## Secure Communication System

### 1. System Architecture

### Components
1. Key Management System (KMS)

Purpose: Manages the creation, exchange, storage, and rotation of cryptographic keys.  
    Responsibilities:
- RSA Key Generation: Generates RSA key pairs (public and private keys) for both Alice and Bob. These keys are used for session key encryption and digital signatures.
- Session Key Generation: Creates a unique AES session key for each communication session. This AES key is used to encrypt the actual message content.
- Key Exchange Management: Handles the secure exchange of public keys and encrypted session keys.
- Key Storage and Rotation: Stores the private RSA keys securely and rotates session keys for each new communication session.

2. Encryption and Decryption Modules

Purpose: Handles encryption and decryption of messages and session keys to ensure message confidentiality.  
    Responsibilities:
- Message Encryption: Encrypts the message content using the AES session key. AES provides fast and secure symmetric encryption for the message body.
- Session Key Encryption: Encrypts the AES session key with the recipient’s RSA public key to ensure only the recipient can access the session key.
- Message Decryption: Decrypts the AES-encrypted message using the decrypted session key.

3. Message Verification Module

Purpose: Verifies the integrity and authenticity of each message.  
Responsibilities:
- Hashing: Creates a SHA-256 hash of the encrypted message to produce a unique message digest.
- Digital Signature: Uses RSA to sign the hash of the message, enabling the recipient to verify that the message was indeed sent by the expected sender.
- Signature Verification: On the recipient’s side, verifies the sender’s digital signature using their public key, confirming that the message hasn’t been altered and originated from the authentic sender.

### Protocols
1. RSA for Key Exchange and Digital Signatures

- Purpose: RSA is used both for securely exchanging the session key and for signing the message to ensure authenticity and non-repudiation.
- Application: 
    - Key Exchange: Alice encrypts the AES session key using Bob’s RSA public key, which only Bob can decrypt with his private key.
    - Digital Signature: Alice signs the message hash with her RSA private key, allowing Bob to authenticate her identity and verify message integrity.

2. AES for Message Encryption

- Purpose: AES is used as a symmetric encryption algorithm to encrypt the actual message content, ensuring confidentiality.
- Application:
    - Alice encrypts her message using the AES session key, providing fast and efficient encryption.
    - Only Bob, after decrypting the AES session key, can decrypt the message content.

3. SHA-256 for Hashing

- Purpose: SHA-256 generates a unique hash for each message, ensuring integrity by detecting any modifications to the message.
- Application:
    - Before signing, Alice creates a SHA-256 hash of the encrypted message. This hash serves as a unique “fingerprint” of the message content.
    - Bob computes the hash of the received message and compares it to the hash extracted from the signature. If the hashes match, the message integrity is confirmed.

### 2. Message Flow Design

#### Step-by-Step Message Encryption Process (Alice to Bob)

Session Key Generation: Alice generates a random AES session key (AES_Session_Key) for symmetric encryption of the message.  
Encrypting the Message: Alice encrypts her message using the AES session key: 
- Encrypted_Message = AES_Encrypt(AES_Session_Key, Message).  
- Session Key Encryption: Alice encrypts the AES_Session_Key with Bob’s public RSA key: 
- Encrypted_Session_Key = RSA_Encrypt(Bob_Public_Key, AES_Session_Key).  
Signing the Message:
- Alice hashes Encrypted_Message using SHA-256 to create a message digest.
- Alice signs this digest with her RSA private key to create a digital signature: Signature = RSA_Sign(Alice_Private_Key, Message_Digest).
- Sending the Message: Alice sends Encrypted_Message, Encrypted_Session_Key, and Signature to Bob.  

#### Step-by-Step Message Decryption Process (Bob receives from Alice)
Decrypting the Session Key: 
- Bob uses his RSA private key to decrypt Encrypted_Session_Key, retrieving the AES_Session_Key.  
  
Decrypting the Message: 
- Bob decrypts Encrypted_Message using AES_Session_Key to obtain the original message.  
  
Verifying the Signature:
- Bob hashes Encrypted_Message to create a digest.
- Bob verifies Signature using Alice’s RSA public key. If the signature matches the hash, the message’s integrity and authenticity are confirmed.

#### Error Handling Procedures
Invalid Signature: 
- If the signature is invalid, Bob discards the message and logs an error.   
   
Decryption Failure: 
- If decryption of the session key or message fails, an error is logged, and Bob requests retransmission from Alice.  
  
Replay Attack Prevention: 
- Alice includes a unique nonce or timestamp with each message. Bob checks this value to confirm it’s recent and unique. If it’s reused, Bob rejects the message.

#### Session Management
Session Key Management: 
- Each communication session uses a unique AES session key, which is discarded at the end of the session.    
  
Long Sessions:  
- For prolonged sessions, the session key is re-generated periodically to maintain security.

#### Flow Diagram:
##### Encryption Flow Diagram (Alice's Side)
![Encryption Flow Diagram - Alice's Side](./encryption.jpg)

##### Decryption Flow Diagram (Bob's Side)
![Decryption Flow Diagram - Bob's Side](./decryption.jpg)


### 3. Security Analysis

Confidentiality
- Achieved: Messages are encrypted using AES, and the AES key is protected with RSA encryption. Only the recipient, who holds the private key, can decrypt the AES session key.

Integrity
- Achieved: The message hash is signed with the sender’s RSA private key, enabling the recipient to confirm that the message hasn’t been altered.

Authenticity and Non-repudiation
- Achieved: The digital signature provides a cryptographic way to verify the sender’s identity, ensuring authenticity. Non-repudiation is ensured as the sender cannot deny sending the message due to their unique digital signature.

Replay Attack Prevention
- Achieved: Using nonces and timestamps allows the system to detect and reject replayed messages.

### 4. Implementation Considerations
- Performance: RSA is used only for encrypting the session key and signing the message. AES is used for the message body to maintain high performance.
- Compatibility: Ensure compatibility of RSA and AES libraries on all devices involved.
- Scalability: Consider a PKI system if the system scales to support additional users beyond Alice and Bob.

### 5. Potential Vulnerabilities and Mitigations
| Vulnerability              | Mitigation                                       |
|----------------------------|--------------------------------------------------|
| Man-in-the-Middle Attack   | Use secure PKI for public key verification       |
| Key Compromise             | Use secure storage, and periodically rotate keys |
| Replay Attacks             | Implement nonce/timestamp-based checks           |
| Brute-force Key Attacks    | Use strong RSA (2048-bit+) and AES (256-bit) keys|
| Side-channel Attacks       | Secure environment, hardware protection          |
